In [ ]:
import numpy as np
print("NumPy:", np.__version__)


In [ ]:
import sys, os
from pathlib import Path

def attach_src_and_cd():
    here = Path.cwd()
    # Busca 'src' en la carpeta actual y en todos los padres
    for base in [here] + list(here.parents):
        cand = base / "src"
        if cand.is_dir():
            if str(cand) not in sys.path:
                sys.path.append(str(cand))
            # Pon el directorio de trabajo en la raíz del repo (donde vive 'src')
            os.chdir(base)
            print("OK: usando src en", cand)
            print("CWD ahora es:", Path.cwd())
            return True
    print("ERROR: no encontré carpeta 'src' empezando desde", here)
    return False

attach_src_and_cd()


## Paso 1: Generamos los puntos sobre una superficie conocida

In [ ]:
from src.data.synth import generate_sphere_points2

N = 600

X = generate_sphere_points2(N)
print("Shape de X:", X.shape)
print("Primeros 5 puntos:\n", X[:5])




# Sanity Check

In [ ]:
import numpy as np

r = np.linalg.norm(X, axis=1)
print("radio medio:", r.mean(), "  desvío:", r.std(), "  min/max:", r.min(), r.max())


# Vizuals
Ya tenemos nuestra esfera. Vamos a verla!

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # activa 3D

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(X[:,0], X[:,1], X[:,2], s=10, alpha=0.7)

ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
ax.set_title("Puntos en la esfera (radio≈1)")
plt.show()


# OPCIONAL RUIDOOOOOOOO

In [ ]:
from src.data.synth import add_gaussian_noise 

X_noisy = add_gaussian_noise(X, std=0.02, seed=42)  # ruido un poco visible


In [ ]:
r_clean = np.linalg.norm(X, axis=1)      # radios ~ 1.000
r_noisy = np.linalg.norm(X_noisy, axis=1)  # radios ~ 0.95–1.05 (aprox)
r_clean[:5], r_noisy[:5]


# KNN

Conectaremos cada punto con su K vecinos mas cercanos

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, csgraph
import matplotlib.pyplot as plt


In [ ]:

from src.graph.knn import build_knn_graph 

k = 20
G = build_knn_graph(X, k)

# 3) Revisa que funcione
print(G)
print("Nodos:", G.number_of_nodes())
print("Aristas:", G.number_of_edges())



Si:
k es muy bajo → analizas la variedad a escalas muy finas.
k es muy alto → analizas la variedad a escalas más gruesas (más “promediadas”).

# Vizual KNN

In [ ]:
from src.viz.plots import plot_knn_graph_3d

plot_knn_graph_3d(
    X,
    G,
    max_edges=500,                  # aumenta/reduce según saturación
    outpath="artifacts/knn_3d.png", # opcional, guarda imagen
    title=f"Grafo k-NN (k={k}, N={N})"
)


# Geodesics

In [ ]:

from __future__ import annotations
import numpy as np
import networkx as nx


In [ ]:
from src.geodesic.shortest_paths import compute_geodesic_distances, handle_disconnections, is_symmetric, has_zero_diagonal


In [ ]:
# Calculamos distancias geodesicas

In [ ]:
D = compute_geodesic_distances(G)
print("Matriz de distancias geodésicas:", D.shape)


In [ ]:
from src.geodesic.shortest_path_FAST import compute_geodesic_distances_fast

# Dijkstra (rápido)
%time D= compute_geodesic_distances_fast(G, method="D")

# (Opcional) si quieres comparar con tu versión vieja:
# %time D_fw = nx.floyd_warshall_numpy(G, weight="weight")


In [ ]:
# Manejamos desconexiones

In [ ]:
D = handle_disconnections(D, strategy="big_value")


In [ ]:
# sanity checks

In [ ]:
print("¿Simétrica?", is_symmetric(D))
print("¿Diagonal cero?", has_zero_diagonal(D))


In [ ]:
# Ejemplo

In [ ]:
print("Distancia entre nodo 0 y 1:", D[19,310])


In [ ]:
# Viz Geodesica

In [ ]:
from src.viz.plots import plot_geodesic_path_3d

In [ ]:
src, dst = 0, 470

In [ ]:
plot_geodesic_path_3d(
    X, G, src, dst,
    outpath="artifacts/geodesic_path.png"
)

In [ ]:
# contrastemos!

In [ ]:
from src.viz.plots import plot_geodesic_vs_euclidean

In [ ]:
plot_geodesic_vs_euclidean(X, G, src, dst, outpath="artifacts/geodesic_vs_euclidean.png")

In [ ]:
#Heatmap


In [ ]:
from src.viz.plots import plot_geodesic_distance_matrix


In [ ]:
plot_geodesic_distance_matrix(
    D
)

In [ ]:
#MDS

In [ ]:
from src.embed.isomap import classical_mds


In [ ]:








#Embeding en 2D

In [ ]:
Y2 = classical_mds(D, n_components=2)
print("Shape embedding 2D:", Y2.shape)


In [ ]:
# Embeding en 3D

In [ ]:
# Embedding en 3D
Y3 = classical_mds(D, n_components=3)
print("Shape embedding 3D:", Y3.shape)

In [ ]:
# Viz 2D

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.scatter(Y2[:,0], Y2[:,1], s=10, alpha=0.6, c="cornflowerblue")
plt.title("ISOMAP Embedding (2D)")
plt.xlabel("Dim 1"); plt.ylabel("Dim 2")
plt.show()


In [ ]:
#Viz 3D

In [ ]:
#Sanity Check

In [ ]:
# Checar media ~ 0 (centrado)
print("Media de cada columna:", Y2.mean(axis=0))

# Ver primeras distancias preservadas
import numpy as np
print("Dist original (0,1):", D[0,1])
print("Dist embebida (0,1):", np.linalg.norm(Y2[0]-Y2[1]))


In [ ]:
# ESTO YA ES ISOMAP COMPLETOOOOO

In [ ]:
# Etiquetemos los puntos para ver en donde se ubican

In [ ]:
n = X.shape[0]
ids = np.arange(n)  # 0..n-1


In [ ]:
# Ejemplo: 25 al azar (reproducible)
np.random.seed(123)
idx_to_label = np.random.choice(n, size=25, replace=False)

# O fija tú los que quieras:
# idx_to_label = np.array([0, 5, 42, 123, 199])


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(X[:,0], X[:,1], X[:,2], s=12, alpha=0.7)

# Etiquetas (solo la muestra)
for i in idx_to_label:
    ax.text(X[i,0], X[i,1], X[i,2], str(i), fontsize=8)

for a in (ax.set_xlim, ax.set_ylim, ax.set_zlim):
    a([-1.2, 1.2])

ax.set_title("Esfera 3D con etiquetas parciales")
plt.savefig("artifacts/fig_sphere_labels.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(Y2[:,0], Y2[:,1], s=20, alpha=0.8)

for i in idx_to_label:
    plt.text(Y2[i,0], Y2[i,1], str(i), fontsize=8)

plt.axis("equal")
plt.title("ISOMAP 2D con etiquetas parciales")
plt.savefig("artifacts/fig_isomap2d_labels.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
# Coloreamos

In [ ]:
# Color por altura Z (del X "limpio", sin ruido, o del Xn)
c = X[:,2]  # o Xn[:,2]

# Esfera
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection="3d")
sc = ax.scatter(X[:,0], X[:,1], X[:,2], c=c, s=12, alpha=0.8)
fig.colorbar(sc, ax=ax, shrink=0.6, label="Z original")
for i in idx_to_label:
    ax.text(X[i,0], X[i,1], X[i,2], str(i), fontsize=8)
plt.savefig("artifacts/fig_sphere_labels_colored.png", dpi=300, bbox_inches="tight")
plt.show()

# ISOMAP 2D
plt.figure(figsize=(6,6))
plt.scatter(Y2[:,0], Y2[:,1], c=c, s=20, alpha=0.9)
for i in idx_to_label:
    plt.text(Y2[i,0], Y2[i,1], str(i), fontsize=8)
plt.axis("equal"); plt.colorbar(label="Z original")
plt.title("ISOMAP 2D con etiquetas y color consistente")
plt.savefig("artifacts/fig_isomap2d_labels_colored.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
# TERMINAMOS ISOMAP PODEMOS AHORA INICIAR LA GEO DESCRIPTIVA

### LPCA (Local PCA) – Estimación de planos tangentes

La idea es que, en una variedad 2D inmersa en \(\mathbb{R}^3\), el vecindario de un punto se puede aproximar por un **plano tangente**.

**Procedimiento:**
1. Tomamos el punto central \(x_c\) y sus vecinos \(x_j\).
2. Centramos: \(P = X_{\text{vecinos}} - x_c\).
3. Hacemos **SVD**: \(P = U \Sigma V^\top\).
4. En \(\mathbb{R}^3\):  
   - Las **dos primeras columnas de \(V\)** forman la base del plano tangente (`E`, de tamaño (3,2)).  
   - La **última columna** es la aproximación al vector normal (`normal`, de tamaño (3,)).  
   - Los **valores singulares principales** (`svals`) indican la variabilidad local en cada dirección tangente.

**Ejemplo de salida:**
- `E.shape = (3,2)` → dos vectores base del plano tangente.  
- `normal.shape = (3,)` → vector normal ortogonal.  
- `svals = [1.29, 0.94]` → varianzas locales en las direcciones tangentes.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


from src.geom.lpca import tangent_plane
from src.viz.plots import plot_tangent_frame_3d 

In [ ]:
center_idx = 200                                 # el punto que quieras analizar
neighbors_idx = np.array(list(G.neighbors(center_idx)))

In [ ]:
center, E, normal, svals = tangent_plane(X, neighbors_idx, center_idx, d=2)
print(E.shape, normal.shape if normal is not None else None, svals)


In [ ]:
plot_tangent_frame_3d(
    X,                     # o X
    center_idx=center_idx,
    E=E,
    normal=normal,
    scale=0.25,
    outpath="artifacts/fig_tangent_center42.png",
    title=f"Plano tangente y normal (idx={center_idx})"
)


In [ ]:
#Vecinos usados

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection="3d")

# todos (suaves)
ax.scatter(X[:,0], X[:,1], X[:,2], s=10, alpha=0.2)

# vecinos resaltados
Nv = X[neighbors_idx]
ax.scatter(Nv[:,0], Nv[:,1], Nv[:,2], s=18)

# centro
c = X[center_idx]
ax.scatter([c[0]], [c[1]], [c[2]], s=60)

plt.savefig(f"artifacts/fig_neighbors_{center_idx}.png", dpi=300, bbox_inches="tight")
plt.show()


Coordenadas locales (LPCA → proyección):

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from src.geom.metric import local_coordinates
from src.viz.plots import plot_tangent_frame_3d

In [ ]:
# Vecinos del punto y proyección al plano tangente
Nv = X[neighbors_idx]                         # (m, 3)
coords2 = local_coordinates(Nv, center=center, E=E)  # (m, 2)

# El propio centro en coords locales (debe ser ~ (0,0))
c_local = local_coordinates(center[None, :], center=center, E=E)[0]
print("Centro en coords locales:", c_local)


In [ ]:
plt.figure(figsize=(5.5, 5.5))
plt.scatter(coords2[:,0], coords2[:,1], s=20, alpha=0.85, label="vecinos")
plt.scatter([c_local[0]], [c_local[1]], s=50, label="centro (0,0)")
plt.axhline(0, linewidth=1); plt.axvline(0, linewidth=1)
plt.gca().set_aspect("equal", adjustable="box")
plt.title(f"Parche local en el plano tangente (idx={center_idx})")
plt.xlabel("e1"); plt.ylabel("e2"); plt.legend()
plt.savefig(f"artifacts/fig_local_patch_{center_idx}.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
pca2 = PCA(n_components=2).fit(coords2)
print("Varianzas locales (patch 2D):", pca2.explained_variance_)

# razón entre la primera y la segunda varianza
ratio = pca2.explained_variance_[0] / pca2.explained_variance_[1]
print("Razón de anisotropía (var1/var2):", ratio)


In [ ]:
plot_tangent_frame_3d(
    X,
    center_idx=center_idx,
    E=E,
    normal=normal,
    scale=0.3,
    outpath=f"artifacts/fig_tangent_frame_{center_idx}.png",
    title=f"Plano tangente y normal en idx={center_idx}"
)


In [ ]:
plot_tangent_frame_3d(
    X, center_idx, E, normal,
    scale=0.3,
    outpath=f"artifacts/fig_tangent_frame_{center_idx}.png",
    title=f"Plano tangente y normal en idx={center_idx}"
)


In [ ]:
#Métrica local en el parche (estimate_metric).

In [ ]:
from src.geom.metric import local_coordinates, estimate_metric
from src.viz.plots import plot_local_patch_with_ellipse

In [ ]:
# 1) Parche local en coords (ya tienes center, E, neighbors_idx, Xn)
Nv = X[neighbors_idx]                         # (m, 3)
coords2 = local_coordinates(Nv, center=center, E=E)  # (m, 2)


In [ ]:
# 2) Estimar métrica (normalizada, trace ≈ 2)
g = estimate_metric(coords2, weighted=False)
print("g:\n", g)
print("trace(g) =", np.trace(g))


In [ ]:
# 3) Diagnóstico rápido: autovalores (anisotropía) y razón
evals, _ = np.linalg.eigh(g)
evals = np.sort(evals)[::-1]  # mayor → menor
print("eigenvals(g) =", evals)
print("razón mayor/menor =", evals[0] / evals[-1])


In [ ]:
# 4) Visual del parche + elipse métrica
plot_local_patch_with_ellipse(
    coords2,
    g,
    outpath=f"artifacts/fig_local_metric_{center_idx}.png",
    title=f'Parche local + elipse métrica (idx={center_idx})'
)


In [ ]:
for center_idx in [5, 100, 101, 69]:
    neighbors_idx = np.array(list(G.neighbors(center_idx)))
    center, E, normal, svals = tangent_plane(X, neighbors_idx, center_idx, d=2)
    coords2 = local_coordinates(X[neighbors_idx], center=center, E=E)
    g = estimate_metric(coords2, weighted=True)

    evals, evecs = np.linalg.eigh(g)              # evals ascendente
    lmin, lmax = float(evals[0]), float(evals[1])
    kappa = lmax / lmin
    tr = lmin + lmax

    print(f"{center_idx} → autovalores g: [{lmin:.4f} {lmax:.4f}]  "
          f"razón(lmax/lmin): {kappa:.2f}  trace: {tr:.4f}")


In [ ]:
# POdemos ver que el punto 3 genero un mucho mejor parche

In [ ]:
# Que pasa si parchamos usando geodesicas y no euclideas? NO es esto contraintuitivo? No perdemos localidad?

In [ ]:
#ULTIMO ANAL. A PARTIR DE LA METRICA ESTIMADA

In [ ]:
### Curvatura local vía parche cuadrático (Monge + mínimos cuadrados)


In [ ]:
from src.geom.curvature_extrinsic import fit_quadratic_patch, curvature_from_quadratic

In [ ]:
# 1) Marco tangente en el punto
center, E, normal, _ = tangent_plane(X, neighbors_idx, center_idx, d=2)


In [ ]:
# 2) Parche local (coords 2D en el plano tangente)
Nv = X[neighbors_idx]                              # vecinos en 3D
coords2 = local_coordinates(Nv, center=center, E=E) # (m,2)

In [ ]:
# 3) Alturas respecto a la normal (para el ajuste h(u,v))
heights = (Nv - center) @ normal                    # (m,)


In [ ]:
# 4) Ajuste cuadrático y curvaturas
coeffs = fit_quadratic_patch(coords2, heights, weighted=True)
k1, k2, K, H, R = curvature_from_quadratic(coeffs)

In [ ]:
print("Coeficientes [a, b, c, α, β, γ]:", coeffs)
print("Curvaturas principales:", k1, k2)
print("Curvatura gaussiana K:", K)
print("Curvatura media H:", H)
print("Curvatura escalar R=2K:", R)

In [ ]:
#Sanity check

In [ ]:
# K y H esperados (esfera unidad, ver teoría abajo sobre signos)
tol = 0.3
print("Esperado esfera: K≈1  | H≈±1 (según orientación de la normal)")

# sanity: magnitudes
assert np.isfinite([k1,k2,K,H,R]).all(), "Aparecieron NaN/Inf en curvaturas"

In [ ]:
from src.eval.curvature import analyze_center

# Ejemplo: esfera unidad con datos Xn y grafo G
_ = analyze_center(center_idx=42, X=X, G=G, weighted=True, radius=1.0, echo=True)
